In [ ]:
# default_exp getCikFilings

# getCikFilings

> Get filings list for a CIK using the SEC's RESTful API.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

from secscan import utils

Download and parse a list of filings for a CIK:

In [ ]:
#export

def cikRestFilingsUrl(cik) :
    return f'/submissions/CIK{str(cik).lstrip("0").zfill(10)}.json'

def appendFilings(res, filingList, startD) :
    fDates = [fDate.replace('-','') for fDate in filingList['filingDate']]
    print(len(fDates),'filings')
    for formType,accNo,fDate in zip(filingList['form'],
                                    filingList['accessionNumber'], fDates) :
        if startD is not None and fDate<startD :
            return True
        res.append((formType,accNo,fDate))
    return False

def getRecent(cik, startD=None, justFirst=True) :
    filingsJson = utils.downloadSecUrl(cikRestFilingsUrl(cik),
                                       restData=True, toFormat='json')['filings']
    res = []
    if appendFilings(res, filingsJson['recent'], startD) or justFirst :
        return res
    print(filingsJson['files'])
    for file in filingsJson.get('files', []) :
        extraList = utils.downloadSecUrl(f'/submissions/{file["name"]}',
                                         restData=True, toFormat='json')
        if appendFilings(res, extraList, startD) :
            break
    return res

Test downloading list of filings for a CIK:

In [ ]:
testF = getRecent(83350)
assert all(tup in testF for tup in (
        ('8-K', '0001437749-21-013386', '20210526'),
        ('10-Q', '0001437749-21-012377', '20210517')
    )),"testing get recent CIK filings on Reserve Petroleum"

In [ ]:
testF = getRecent(732717, startD='2017', justFirst=False)
assert all(tup in testF for tup in (
        ('4', '0001127602-22-020264', '20220802'),
        ('425', '0001193125-17-000682', '20170103')
    )),"testing get recent CIK filings on AT&T"

1023 filings
[{'name': 'CIK0000732717-submissions-001.json', 'filingCount': 2018, 'filingFrom': '2013-02-04', 'filingTo': '2019-02-06'}, {'name': 'CIK0000732717-submissions-002.json', 'filingCount': 2019, 'filingFrom': '2007-02-02', 'filingTo': '2013-02-03'}, {'name': 'CIK0000732717-submissions-003.json', 'filingCount': 2001, 'filingFrom': '1994-05-19', 'filingTo': '2007-02-01'}, {'name': 'CIK0000732717-submissions-004.json', 'filingCount': 7, 'filingFrom': '1994-02-14', 'filingTo': '1994-05-18'}]
2018 filings


In [ ]:
#hide
# uncomment and run to regenerate all library Python files
# from nbdev.export import notebook2script; notebook2script()